# All Pairs Shortest Paths (APSP)

## Problem Definition

Input: A directed graph G with edges with lengths Ce. Edges can have negative lengths too. No distinguished source

Goal: Compute thel ength of a shortest u -> v path for all pairs of vertices u,v. 
 - Or, correctly report that G has a negative cycle. 
 
Suppose we have something that solves a single-source shortest path subroutine. How many times do we need to invoke this to solve the all-pairs shortest paths?
 - Need to run it a total of n times. Designate arbitrary vertex s, will compute shortest path distances from that choice of S to all destinations. Computes N shortest paths
     - In total, have n^2 pairs of shortest paths. So, run n times, iterating over all choices. Have shortest path for all pairs. 
     

Case with Non-negative Edge Costs:
 - Determines which singe-source path subroutine to use. If all non-negative, can use Dijkstras O(mlogn)
     - So, will use total of O(nmlogn). In sparse graph, has O(n^2logn) (m = O(n)). In dense, has O(n^3logn) (m = O(n^2)). 
         - For dense graph, not sure if there is anything fundamentally faster than cubic time. 
         - Could argue that there are quadratic # of shortest paths to compute, for given pair u,v, may have up to linear number of edges. Thus, may always require cupic time. But, this is by no means a proof and very wishy washy. Still unclear. 
         - Remember, matrix multiplication can be done faster than naive cubic time algorithm. Can decompose. 

General Case with Negative Edges Allowed:
 - Resort to Bellman-Ford algorithm instead. n * Bellman-Ford = n * O(nm) = O(n^2m). 
     - Sparse, then O(n^3). If dense, then O(n^4). Jesus fuck. This is ass. 
     - Fix with Floyd-Warshall to be discussed.

## Optimal Substructure

Floyd-Warshall Algorithm Introduction:
 - Solves APSP in O(n^3) time even with negative edge lengths. 
 - At least s good as n Bellman-Fords, better in dense graphs
 - In graphs with nonnegative edge costs, competitive with n Dijkstra's in dense graphs. 
     - In practice, code both up and see which one runs better on your particular graph. 
 - Special Case: Transitive closure of a binary relation i.e. all-pairs rechability (without caring abt actual shortest path)
     - Can be done with this with some modifications to be more efficient. 
 - Open Question: Can we do better than O(n^3) in dense graphs? Not sure. 

**Optimal Substructure**

Recall: Can be tricky to define ordering on subproblems in graph problems. Bellman-Ford used i, # of edges allowed in a shortest path that naturally created an order. Do something similar but more stringent here. 

Key Idea: Restrict both # of vertices allowed to use in shortest path *and* which vertices are allowed to be used. 
 - Order the vertices arbitrarily for V = {1,2,3,4...n}
 - Let Vk = {1,2,3...k}, vertices 1 through k. 
 - Lemma: Suppose G has no negative cycles. 
     - Fix source i in V, destination j in V, and k in {1,2,3...k}. k in between i and j, interal nodes. Will produce n^3 subproblems. 
     - P = shortest cycle-free i-j path with all internal nodese in Vk. 
     - Example: i = 17, j = 10, k = 5 (allowed to use 1,2,3,4,5). Let's say some optimal path goes through 7, not allowed to look at bc only allowed to use k. 
 - Full Lemma: Suppsoe G has no negative cost cycle. Let P be a shortest (cycle-ree) i-j path with all internal nodes in V(k). Then:
     - Case1: If path P does not include k, then P is a shortest (cycle-free) i - j path with all internal vertices in Vk-1.
     - Case 2: Suppose P uses k in the middle. Then can consider P in two subpaths:
         - P1 = i to k path; P2 = k to j path.
         - Internal to P1 and P2, only includes nodes from 1 to k-1. 
         - P1 and P2 are optimal solutions as well between i to k and k to j respectively. 
         - Proof of correctness is similar to previous proofs. Same for P1 and P2
             - Suppose there was P1' that was more optimal than P1. But then, P = P1 + P2 > P1' + P2, which contradicts optimality of P. Thus, this msut be true. 

## Floyd-Warshall Algorithm

Algorithm:
 - Setup: Let A = 3D array indexed by i, j, k.
 - A[i,j,k] = length of a shortest i-j path with all internal nodes in {1,2,3...k}
 - For all k = 0, 
     - When i = j, A[i,j,0] = 0
     - When (i,j) is an Edge (i.e. no internal nodes), A[i,j,0] = Cij (cost of edge)
     - When (i,j) not an Edge, A[i,j,0] = infinity. Cannot get. 
 - For k in range (1,n)
     - For i in range (1,n), order with j irrelevant 
         - For j in range (1,n):
             - A[i,j,k] = min(
                 - Case 1: A[i,j,k-1]
                 - Case 2: A[i,k, k-1] + A[k,j,k-1]) 
             - Will already have solutions from previous subproblems
 - Correctness shown via optimal subproblem lemma
 - Running Time: O(n^3) subproblems, each subproblem takes O(1) per subproblem. So, total O((n^3). 

Questions:
 - 1. What if there is a negative cycle? Remember, subproblem lemma assumed no negative cycle. 
     - Scan the diagonal of the final numbers. Will have A[i,i,n] < 0 for at least one i in V at the end of the algorithm
         - If see a negative number here, a negative cycle exists.
             - Then after enough allowed k's, i to i will run through this negative cycle. This will decrease the value of A[i,i,k] for whichever k this is true. 
             - Remember, takes min A[i,k, k-1] + A[k, j, k-1].
             - In this case, will be A[i, k, k-1] + A[k, i, k-1]. These are basically the two halves of the cycle (with k also in the cycle). If a negative cycle, this will be < 0. 
         - If diagonal all zeros, return final batch of A[i,j,n] as correct shortest paths.
 - 2. How do we reconstruct a shortest i to j path?
     - Need store extra information, constant info per subproblem. 
     - Subproblems indexed by origins and destinations. For each choice of origin and destination, remember some other vertex on a shortest path. 
         - Remember B[i,j] = the highest index vertex on shortest path from i to j. 
         - Computing in forward direction:
             - If Case 2, set B[i,j] = k on forward pass. Remember last vertex responsible for changing shortest path dist from i to j.
         - Reconstructing (assuming accurate B[i,j] calculation):
             - Let's say source 23, destination 17. 
                 - Look at B[23, 17] = 43
                 - Then look at B[23, 43] ... and so on. Keep track of all of these until you get to B[23, 23]. Shortest path at most n vertices, figuring out one vertex per recursive call. 

## A Reweighting Technique (Johnson's Algorithm)

Reecall: still observing APSP problem which reduces to n invocations of SSSP (single-source shortest path)
 - Nonnegative edge lengths: 
     - Dijkstra's, runs O(nmlogn). 
 - General Edge Lengths: O(mn^2) via Bellman-Ford (not relevant)
     - Floyd-Warshall runs O(n^3)
 - Johnson's Algorithm For General Edge Lengths):
     - Reduces APSP to:
         - 1 invocation of Bellman-Ford (O(mn))
         - n invocations of Dijkstra (O(nmlogn)), dominates just barely.
         - Total O(nmlogn)
     - Recall in Dijkstra's, considered adding "most-negative value" of a negative edge to all edge-lengths 
         - Problem is, only preserves shortest paths if all s-t paths of G have same number of edges. Adding to all adds to paths differentially based on # of edges in path, ofc. 
     - Consider instead, G is directed graph with general edge lengths Ce. Fix a real number Pv for each vertex v
         - For every edge (e,v) of G, Ce' = ce + pu - pv. Transforming edge lengths. u is tail, v is head. 
             - EX: pu = -4, pv = -3, Cuv = 2. C' = 2 + (-4) - (-3) = 1.
         - Consider path P from s to t with length L with original edge lengths ce. What is P's new length with edge lengths Ce'? P' = L + ps - pt. Everything in the middle cancels out. 
             - So, reweighting using vertex weights {pv} adds the same amount (namely, (ps - pt)) to every s - t path. 
             - Node reweighting shifts every path from s to t by exacatly the same amount. Preserves shortest path. 
             - Key: Need to construct vertex weights then that change arbitrary instance that has negative edge lengths into one with nonnegative lengths. Transform a general graph then run Dijkstra's n times. Nice. 

Uses Bellman-Ford to calculate vertex reweighting to make all edges nonnegative. 
 - Idea is using subroutine in single-source shortest path algo. Need a source vertex
     - When picking arbitrary source though, may not be able to reach all other vertices. To gt weights, need to reach all (to obtain a finite path length).
     - Instead, add a new vertex s that connects to all other vertices with a drect arc length of 0. 
     - Compute shortest paths from s to all vertices from original graph. Will get finite shortest path distance to others using Bellman-Ford (bc negative edges) 
         - s also invisible to original vertices in G bc no edges going to s. 
     - All shortest paths from s to other v will be <= 0 (because initiated with 0). 
         - These computed shortest paths are exactly the vertex reweights we want. 
         - i.e., pv = length of a shortest s -> v path. 
     - Do transformations as outlined above for all edges. Can run Dijkstra's n times to compute all shortest path distances. 

Johnsons:
 - Input: Directed graph G, general edge lengths Ce
 - Form G' by adding a new vertex s and a new edge (s,v) for with length 0 for each v in G
 - Run Bellman-Ford on G' with source vertex s.
     - If detects negative cycle, report it and break out. This is because this cycle must be in original graph G since new vertex s has no incoming arcs and cannot create cycle. No cycle then continue
 - Assign pvs based on shortest path from p to v; recalculate edge lengths for all edges e(u,v) so C'e = Ce + pu - pv. 
 - for each vertex u of G:
     - Dijkstra's algo in G with edge lengths (C'e) with source vertex u to compute shortest-path dist d'(u,v) for each v in G. 
     - Note that d' are shortest-paths with respect to modified lengths. Need original true distances with respect to original lengths.
     - d' = d + pu - pv. So, d = d' + pv - pu, do this for each pair (u,v) in G. Return the new d(u,v)'s. 
     
 - RunTime: 
     - Adds 1 vertex and n new edges, O(n)
     - Bellman-Ford, O(mn) time
     - Compute modified costs, constant time per edge, O(m)
     - Dijkstra's n times, so O(nmlogn)
     - For each pair of vertexs, constant work to reset, O(n^2)
     - Dijkstra's n times donimantes, so in total O(mnlogn). Note, m = Omega(n) i.e. m >= n-1
 - For sparse graphs, this is a lot better
     - Compared to Floyd Warshall, O(n^3) or for n-Bellman-Fords O(n^3) as well
 - Matches running time we were already getting in special case when edge links were non negative, very cool. 
 
Correctness:
 - Based primarily on why modified edge-lengths are non-negative. If this is proven, then correctness is done (as previously shown that reweighted edges preserves shortest path and that paths change by common amount (pu - pv)). 
 - Claim: for every edge e of G, the reweighted length C'e is nonnegative
 - Proof: Fix an edge (u,v)
     - By construction, pu = length of shortest s - u apth in G'; pv = length of shortest s-v path of G'
     - Let P = a shortest s-u path in G' (with length pu)
         - P + (u,v) = an s-v path wih length pu + Cuv. 
         - pv must be <= P + (u,v) i.e. <= pu + Cuv.
         - C'uv = cuv + pu - pv >= 0 (via algebra from above). QED. 